In [110]:
import pandas as pd
import nltk
from cleantext import clean
import tqdm

## Data importation

In [5]:
reviews_df = pd.read_csv("data/reviews.csv", sep=";")

In [9]:
reviews_df

,category_link,company_link,company_name,score,description,review_page_nb,review_link,review_score,review_title,review_text,category
0,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/65a5388a60d6...,5,Application conviviale pour passer ses…,Application conviviale pour passer ses command...,food_beverages_tobacco
1,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/65a53245a223...,5,Très facile pour la commande,"Très facile pour la commande, très rapide et l...",food_beverages_tobacco
2,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/659f0e15dd57...,5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir ...",food_beverages_tobacco
3,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/65a535a6ef47...,5,Livraison parfaite !!,Livraison parfaite !!! Tout est clair et respe...,food_beverages_tobacco
4,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/659e5c5b3f1e...,5,Livraison super efficace,"Livraison super efficace, produits de qualités...",food_beverages_tobacco
...,...,...,...,...,...,...,...,...,...,...,...
235498,https://fr.trustpilot.com/categories/vehicles_...,https://fr.trustpilot.com/review/allpneus.com,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",76,https://fr.trustpilot.com/reviews/659c57814912...,1,non conforme au descriptif du site,après une première livraison fausse : pas les ...,vehicles_transportation
235499,https://fr.trustpilot.com/categories/vehicles_...,https://fr.trustpilot.com/review/allpneus.com,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",76,https://fr.trustpilot.com/reviews/6592e5f3cf8b...,4,presque parfait !! manque la facture !!,"commande et livraison ok, paiement par carte i...",vehicles_transportation
235500,https://fr.trustpilot.com/categories/vehicles_...,https://fr.trustpilot.com/review/allpneus.com,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",76,https://fr.trustpilot.com/reviews/65781d1656c0...,4,Parfait sauf le service de livraison,Rien a dire pour ALLPNEUS c'est parfait. Comma...,vehicles_transportation
235501,https://fr.trustpilot.com/categories/vehicles_...,https://fr.trustpilot.com/review/allpneus.com,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",76,https://fr.trustpilot.com/reviews/658587017947...,5,j'ai trouvé chez vous la dimension peu…,j'ai trouvé chez vous la dimension peu courant...,vehicles_transportation


## Text Data Cleaning

In [11]:
reviews_df = reviews_df[["company_name", "score", "description", "review_score", "review_title", "review_text", "category"]]

In [12]:
reviews_df

,company_name,score,description,review_score,review_title,review_text,category
0,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,5,Application conviviale pour passer ses…,Application conviviale pour passer ses command...,food_beverages_tobacco
1,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,5,Très facile pour la commande,"Très facile pour la commande, très rapide et l...",food_beverages_tobacco
2,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir ...",food_beverages_tobacco
3,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,5,Livraison parfaite !!,Livraison parfaite !!! Tout est clair et respe...,food_beverages_tobacco
4,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,5,Livraison super efficace,"Livraison super efficace, produits de qualités...",food_beverages_tobacco
...,...,...,...,...,...,...,...
235498,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",1,non conforme au descriptif du site,après une première livraison fausse : pas les ...,vehicles_transportation
235499,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",4,presque parfait !! manque la facture !!,"commande et livraison ok, paiement par carte i...",vehicles_transportation
235500,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",4,Parfait sauf le service de livraison,Rien a dire pour ALLPNEUS c'est parfait. Comma...,vehicles_transportation
235501,Allpneus - APN Pneumatiques,"4,6","\nBienvenue chez Allpneus !\nDepuis 2006, ALLP...",5,j'ai trouvé chez vous la dimension peu…,j'ai trouvé chez vous la dimension peu courant...,vehicles_transportation


cleantext library to remove emojis and line breaks

In [30]:
reviews_df["c"].values[0]

'Le Fourgon vous livre vos boissons consignées à domicile :\n🍶 la commande se passe sur lefourgon.com : bières, jus, sodas, eaux, lait, vins, soupes, spririteux, & co.\n🚚 nous vous livrons gratuitement chez vous sur le créneau choisi\n🌱 et au passage suivant on récupère vos bouteilles vides qu’on renvoie lavées au producteur pour ré-emploi #zerodechet'

In [32]:
clean(reviews_df["description"].values[0], no_emoji=True, no_line_breaks=True, lower=False).replace("#", "")

"Le Fourgon vous livre vos boissons consignees a domicile : la commande se passe sur lefourgon.com : bieres, jus, sodas, eaux, lait, vins, soupes, spririteux, & co. nous vous livrons gratuitement chez vous sur le creneau choisi et au passage suivant on recupere vos bouteilles vides qu'on renvoie lavees au producteur pour re-emploi zerodechet"

In [33]:
reviews_df["description"] = reviews_df["description"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))

C:\Users\marya\AppData\Local\Temp\ipykernel_31708\2730620396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df["description"] = reviews_df["description"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))


In [35]:
reviews_df["description"].values[100]

'"Belleville Brulerie - Paris" est une torrefaction de cafes de qualite en faveur du cafe frais, du cafe filtre, du cafe bodum et des assemblages.'

In [36]:
reviews_df["review_title"] = reviews_df["review_title"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))

C:\Users\marya\AppData\Local\Temp\ipykernel_31708\3339734242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df["review_title"] = reviews_df["review_title"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))


In [37]:
reviews_df["review_text"] = reviews_df["review_text"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))

C:\Users\marya\AppData\Local\Temp\ipykernel_31708\864621105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df["review_text"] = reviews_df["review_text"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))


In [38]:
reviews_df

,company_name,score,description,review_score,review_title,review_text,category
0,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Application conviviale pour passer ses,Application conviviale pour passer ses command...,food_beverages_tobacco
1,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Tres facile pour la commande,"Tres facile pour la commande, tres rapide et l...",food_beverages_tobacco
2,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Premiere experience reussie !,"Pour nous, c'etait une premiere.Ravis d'avoir ...",food_beverages_tobacco
3,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Livraison parfaite !!,Livraison parfaite !!! Tout est clair et respe...,food_beverages_tobacco
4,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignees ...,5,Livraison super efficace,"Livraison super efficace, produits de qualites...",food_beverages_tobacco
...,...,...,...,...,...,...,...
235498,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",1,non conforme au descriptif du site,apres une premiere livraison fausse : pas les ...,vehicles_transportation
235499,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",4,presque parfait !! manque la facture !!,"commande et livraison ok, paiement par carte i...",vehicles_transportation
235500,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",4,Parfait sauf le service de livraison,Rien a dire pour ALLPNEUS c'est parfait. Comma...,vehicles_transportation
235501,Allpneus - APN Pneumatiques,"4,6","Bienvenue chez Allpneus ! Depuis 2006, ALLPNEU...",5,j'ai trouve chez vous la dimension peu,j'ai trouve chez vous la dimension peu courant...,vehicles_transportation


In [39]:
# reviews_df.to_csv("data/reviews_cleaned_1.csv", sep=";", index=False)

## Translate text

does not work...

In [53]:
from translate import Translator

In [99]:
translator = Translator(from_lang="fr", to_lang="en")
translation = translator.translate("Non conforme au descriptif du site")
translation

'Not in accordance with the site description'

In [71]:
from translate import Translator
translator= Translator(to_lang="fr")
translation = translator.translate("This is a pen.")
translation

"C'est un stylo"

works

In [77]:
from deep_translator import GoogleTranslator

# Use any translator you like, in this example GoogleTranslator
translated = GoogleTranslator(source='auto', target='de').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig

In [78]:
translated

'Mach weiter so, du bist großartig'

In [81]:
translated = GoogleTranslator(source='fr', target='en').translate("non conforme au descriptif du site")

In [82]:
translated

'not consistent with the site description'

In [89]:
company_df = reviews_df[["company_name", "score", "description"]].drop_duplicates(subset=["company_name"], ignore_index=True)

In [93]:
company_df["description_len"] = company_df["description"].str.len()

In [109]:
company_df[company_df["description_len"] >= 5000] 

,company_name,score,description,description_len
708,educastream.com,"3,5","Haec igitur Epicuri non probo, inquam. De cete...",8482
1785,ScanMarine TM (Cofrance SARL),"3,8",". SCANMARINE , . . The leading yacht broker in...",7586
3230,EDF ENR,"3,3","Avis sur EDF ENR Filiale du groupe EDF, nous c...",9809
6811,Sofamobili,"1,2","Le 01/08/2017 Le 01/08/2017 Cher Client, Je fa...",5167
7104,ethnik-shop.com,"4,2","Ethnik-Shop, notre Boutique specialisee dans l...",5031
8441,HCOMMEHOME,"2,1","Le 01/08/2017 Cher Client, Je fais suite a vot...",5153


In [97]:
# company_df.to_csv("data/company_desc.csv", sep=";", index=False)

In [91]:
company_df["description_en"] = company_df["description"].apply(lambda x: GoogleTranslator(source='fr', target='en').translate(x))

NotValidLength: Haec igitur Epicuri non probo, inquam. De cetero vellem equidem aut ipse doctrinis fuisset instructior est enim, quod tibi ita videri necesse est, non satis politus iis artibus, quas qui tenent, eruditi appellantur aut ne deterruisset alios a studiis. quamquam te quidem video minime esse deterritum. Homines enim eruditos et sobrios ut infaustos et inutiles vitant, eo quoque accedente quod et nomenclatores adsueti haec et talia venditare, mercede accepta lucris quosdam et prandiis inserunt subditicios ignobiles et obscuros. Homines enim eruditos et sobrios ut infaustos et inutiles vitant, eo quoque accedente quod et nomenclatores adsueti haec et talia venditare, mercede accepta lucris quosdam et prandiis inserunt subditicios ignobiles et obscuros. Constituendi autem sunt qui sint in amicitia fines et quasi termini diligendi. De quibus tres video sententias ferri, quarum nullam probo, unam, ut eodem modo erga amicum adfecti simus, quo erga nosmet ipsos, alteram, ut nostra

In [101]:
translator = Translator(from_lang="fr", to_lang="en")
translation = translator.translate("Non conforme au descriptif du site")
translation

'Not in accordance with the site description'

In [105]:
translated = company_df[company_df["description_len"] > 5000]["description"].apply(lambda x: translator.translate(x))

In [108]:
translated.values

array(['QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS',
       'QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEEDED. MAX ALLOW

In [115]:
translator = GoogleTranslator(source='fr', target='en')
# we want to translate all the contents of the "description" column except the ones that are longer than 4999 character
# because the Google Translator API does not support more than 5000 characters
# we will translate the longer ones later
desc_en = []
for desc in tqdm(company_df["description"]):
    if len(desc) > 5000:
        sentences = desc.split(".")
        translated = ""
        for sentence in sentences:
            translated_sentence = translator.translate(sentence)
            if translated_sentence != None:
                translated += translated_sentence + ". " # we add the "." separator back
        desc_en.append(translated)
    else:
        desc_en.append(translator.translate(desc))

In [116]:
desc_en

['Haec igitur Epicuri non probo, inquam. De cetero vellem equidem aut ipse doctrinis futset instructior est enim, quod tibi ita videri necesse est, non satis politus iis artibus, quas qui tenent, eruditi appellantur aut ne deterruisset alios a studiis. quamquam te quidem video mini esse deterritum. Homines enim eruditos et sobrios ut infaustos et inutiles vitant, eo quoque accedente quod et nomenclatores adsueti haec et talia venditare, mercede accepted lucris quosdam et prandiis insert subditicios ignobiles et obscuros. Homines enim eruditos et sobrios ut infaustos et inutiles vitant, eo quoque accedente quod et nomenclatores adsueti haec et talia venditare, mercede accepted lucris quosdam et prandiis insert subditicios ignobiles et obscuros. Constituendi autem sunt qui sint in amicitia fines et quasi termini diligendi. De quibus tres video sentias ferri, quarum nullam probo, unam, ut eodem modo erga amicum adfecti simus, quo erga nosmet ipsos, alteram, ut nostra in amicos benevolenti